# 1. Введение

### О ЧЁМ БУДЕМ ГОВОРИТЬ?

Каждый раз, когда мы обсуждали хранящиеся в рабочих таблицах данные, мы обязательно указывали их принадлежность к тому или иному типу.

Прежде мы не акцентировали на этом внимание, но в действительности знание этих типов крайне важно для всего рабочего процесса. И этот модуль мы посвятим разговору о типах данных, а также об особенностях взаимодействия с некоторыми из них.